<a href="https://colab.research.google.com/github/Ads-cmu/NLPProject/blob/main/QG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Handle non ascii characters and utf characters later on

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 18.4 MB/s 
     |████████████████████████████████| 7.6 MB 48.7 MB/s 
     |████████████████████████████████| 182 kB 10.4 MB/s 


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 12.5 MB/s 


In [ ]:
!git lfs install
!git clone https://huggingface.co/mrm8488/t5-base-finetuned-question-generation-ap

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 't5-base-finetuned-question-generation-ap'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 43 (delta 17), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (43/43), done.
tcmalloc: large alloc 1471086592 bytes == 0x55cf6a800000 @  0x7fb6174912a4 0x55cf2e19578f 0x55cf2e1728db 0x55cf2e1275b3 0x55cf2e0cb34a 0x55cf2e0cb806 0x55cf2e0e8ad1 0x55cf2e0e9069 0x55cf2e0e9593 0x55cf2e18e482 0x55cf2e02ecc2 0x55cf2e015a75 0x55cf2e016735 0x55cf2e01573a 0x7fb6167d8c87 0x55cf2e01578a
tcmalloc: large alloc 1471086592 bytes == 0x55cf6a800000 @  0x7fb6174912a4 0x55cf2e19578f 0x55cf2e1728db 0x55cf2e1275b3 0x55cf2e0cb34a 0x55cf2e0cb806 0x55cf2e0e8ad1 0x55cf2e0e9069 0x55cf2e0e9593 0x55cf2e18e482 0x55cf2e02ecc2 0x55cf2e015a75 0x55cf2e

In [ ]:
from collections import defaultdict
import spacy
from spacy.tokens import DocBin
import os
import sys
import logging
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [ ]:
logging.getLogger("transformers").setLevel(logging.ERROR)
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nlp = spacy.load("en_core_web_sm")

#import neuralcoref
#neuralcoref.add_to_pipe(nlp)

In [ ]:
class QGPipeline:
    def __init__(self, input_file_path, num_of_questions):
        self.document = self.read_doc(input_file_path)
        doc = self.document#nlp(self.document)
        coref_doc = doc#doc._.coref_resolved
        coref_doc = self.preprocess_doc_remove_sections(coref_doc)
        doc = nlp(coref_doc)
        sentences = [sent for sent in doc.sents]
        ner_dict = self.ner(sentences)
        self.avg_confidence = self.generate_n_questions(num_of_questions, ner_dict, sentences)

    def read_doc(self, doc_path):
        with open(doc_path) as f:
            document = f.read()
            return document
    
    def preprocess_doc_remove_sections(self, text):
        return " ".join(text.split())

    def ner(self, sentences):
        ner_dict = defaultdict(list)
        for idx, sentence in enumerate(sentences):
            for entity in sentence.ents:
                ner_dict[entity.label_].append((idx, str(entity.text)))
        return ner_dict

    def generate_question(self, answer, context, max_length=64):
        input_text = f"answer: {answer}  context: {context} </s>"
        features = tokenizer([input_text], return_tensors='pt')
        output = model.generate(input_ids=features['input_ids'], attention_mask=features['attention_mask'], max_length=max_length)
        question = tokenizer.decode(output[0])[16:-4]
        return question

    # def get_subject_phrase(self, context):
    #     for token in context:
    #         if ("nsubj" in token.dep_):
    #             subtree = list(token.subtree)
    #             start = subtree[0].i
    #             end = subtree[-1].i + 1
    #             return context[start:end]
    
    def get_object_phrase(self, context):
        for token in context:
            if ("dobj" in token.dep_):
                subtree = list(token.subtree)
                start = subtree[0].i
                end = subtree[-1].i + 1
                return context[start:end]

    def getScore(self, visited_set, idx, ner, question, context, confidence_score):
        score = 0
        words = question.split(' ')
        if(len(words) < 4):
            score-=5
        if(words[-1].lower() == 'and?'):
            score-=5
        q = nlp(question)
        if(len(q.ents) == 0):
            score-=3
        if((idx, ner) in visited_set): #downrank questions that have been generated from the same context
            score-=1
        if words[0] not in {'What','When','How',"Where",'Why','Who'}: 
            score-=5
        if self.get_object_phrase(context) is None: #questions that are formed from context without dobj
            score-=1
        if confidence_score < 0.8:
            score-=1
        if confidence_score < 0.4:
            score-=3
        '''
        non_fillers = [word.replace('?','') for word in words if len(word)>4]
        if(len(set(non_fillers))!=len(non_fillers)):
            score-=1
        '''
        return score
    def generate_n_questions(self, n, ner_dict, sentences):
        results = []
        discarded_candidates=[]
        count = 0
        visited_set=set()
        for ner in ['PERSON', 'LOC','GPE', 'TIME', 'ALL']:
            list_ner = ner_dict[ner]
            for idx,entity in list_ner:
                question = self.generate_question(entity, sentences[idx])
                confidence_score = qa_model(question=str(question), context=str(sentences[idx]))['score']
                score = self.getScore(visited_set, idx, ner, question, sentences[idx], confidence_score)
                if(score>=0):
                    print(question)
                    results.append(confidence_score)
                    # print(sentences[idx])
                    # print(confidence_score)
                    # print(self.get_object_phrase(sentences[idx]))
                    count+=1
                else:
                    discarded_candidates.append([score, question,confidence_score])
                visited_set.add((idx,ner))
                if count == n:
                    return sum(results)/n
        discarded_candidates = sorted(discarded_candidates,key=lambda x:x[0],reverse=True)
        
        i = 0
        while(count < n and i < len(discarded_candidates)):
            print(discarded_candidates[i][1])
            results.append(discarded_candidates[i])
            count+=1
            i+=1
        
        while(count<n):
            print("what is a question?")
            count+=1
        return sum(results)/n

In [ ]:
document = "/content/drive/MyDrive/Courses/NLP/Data/pikachu_pokemon.txt"
qa_model = pipeline("question-answering")
qg_pipeline = QGPipeline(document, 50)
print(qg_pipeline.avg_confidence)

Who was the developer of Pikachu?
What game features a Pikachu as the only available Starter Pokémon?
What color is Pokémon Let's Go based on?
What game has Pikachu in almost all levels?
Who is the protagonist of the first episode of Pokémon?
Who puts himself in danger to defend Pikachu?
Who is Pikachu from the anime series?
Who was the mayor of Topeka in 1998?
What was the name of the first Pokémon jet?
What game was released in October of 1996?
Who called Pikachu the best Electric-type Pokémon?
What did the team say would be the most popular Pokémon?
Who said that Pikachu was the most important Pokémon in the franchise?
Who said that Pikachu is the best animal companion in video games?
Who said Pikachu was the third favorite Pokémon?
What character was named one of the 12 most influential video game characters of all time by Time Magazine?
Who included Pikachu on their "30 best heroes of the last 30 years" list?
Who wrote about Pikachu in 2021?
Who criticized Pikachu's over-represent

In [ ]:
getScore("What dynasty was the name of the royal dynasty?")

What dynasty was the name of the royal dynasty?


-1

In [ ]:
import sys
import spacy
import codecs
from collections import defaultdict
from spacy.tokens import DocBin
import os
import logging
from import_QG import tokenizer, model, summarizer, qa_model

import re

nlp = spacy.load("en_core_web_sm")

import neuralcoref
neuralcoref.add_to_pipe(nlp)

class QGPipeline:
    def __init__(self, input_file_path, num_of_questions):
        self.document = self.read_doc(input_file_path)
        doc = nlp(self.document)
        coref_doc = doc._.coref_resolved
        coref_doc = self.preprocess_doc_remove_sections(coref_doc)
        doc = nlp(coref_doc)
        sentences = [sent for sent in doc.sents]
        ner_dict = self.ner(sentences)
        self.generate_n_questions(num_of_questions, ner_dict, sentences)

    def read_doc(self, doc_path):
        with open(doc_path) as f:
            document = f.read()
            return document
    
    def preprocess_doc_remove_sections(self, text):
        return " ".join(text.split())

    def ner(self, sentences):
        ner_dict = defaultdict(list)
        for idx, sentence in enumerate(sentences):
            for entity in sentence.ents:
                ner_dict[entity.label_].append((idx, str(entity.text)))
        return ner_dict

    def generate_question(self, answer, context, max_length=64):
        input_text = f"answer: {answer}  context: {context} </s>"
        features = tokenizer([input_text], return_tensors='pt')
        output = model.generate(input_ids=features['input_ids'], attention_mask=features['attention_mask'], max_length=max_length)
        question = tokenizer.decode(output[0])[16:-4]
        return question

    # def get_subject_phrase(self, context):
    #     for token in context:
    #         if ("nsubj" in token.dep_):
    #             subtree = list(token.subtree)
    #             start = subtree[0].i
    #             end = subtree[-1].i + 1
    #             return context[start:end]
    
    def get_object_phrase(self, context):
        for token in context:
            if ("dobj" in token.dep_):
                subtree = list(token.subtree)
                start = subtree[0].i
                end = subtree[-1].i + 1
                return context[start:end]

    def getScore(self, visited_set, idx, ner, question, context, confidence_score):
        score = 0
        words = question.split(' ')
        if(len(words) < 4):
            score-=5
        if(words[-1].lower() == 'and?'):
            score-=5
        q = nlp(question)
        if(len(q.ents) == 0):
            score-=3
        if((idx, ner) in visited_set): #downrank questions that have been generated from the same context
            score-=1
        if words[0] not in {'What','When','How',"Where",'Why','Who'}: 
            score-=5
        if self.get_object_phrase(context) is None: #questions that are formed from context without dobj
            score-=1
        if confidence_score < 0.8:
            score-=1
        if confidence_score < 0.4:
            score-=3
        '''
        non_fillers = [word.replace('?','') for word in words if len(word)>4]
        if(len(set(non_fillers))!=len(non_fillers)):
            score-=1
        '''
        return score

    def generate_n_questions(self, n, ner_dict, sentences):
        discarded_candidates=[]
        count = 0
        visited_set=set()
        for ner in ['PERSON', 'LOC','GPE', 'TIME', 'ALL']:
            list_ner = ner_dict[ner]
            for idx,entity in list_ner:
                question = self.generate_question(entity, sentences[idx])
                confidence_score = qa_model(question=str(question), context=str(sentences[idx]))['score']
                score = self.getScore(visited_set, idx, ner, question, sentences[idx], confidence_score)
                if(score>=0):
                    print(question)
                    # print(sentences[idx])
                    # print(confidence_score)
                    # print(self.get_object_phrase(sentences[idx]))
                    count+=1
                else:
                    discarded_candidates.append([score, question])
                visited_set.add((idx,ner))
                if count == n:
                    return
        discarded_candidates = sorted(discarded_candidates,key=lambda x:x[0],reverse=True)
        
        i = 0
        while(count < n and i < len(discarded_candidates)):
            print(discarded_candidates[i][1])
            count+=1
            i+=1
        
        while(count<n):
            print("what is a question?")
            count+=1

if __name__ == "__main__":    
    input_file_path = sys.argv[1]
    num_of_questions = int(sys.argv[2])
    qg_pipeline = QGPipeline(input_file_path, num_of_questions)